In [ ]:
!pip install --upgrade jax jaxlib

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np

# Define the data directories
train_dir = "/content/drive/MyDrive/train"
test_dir = "/content/drive/MyDrive/test"

# Define the image data generators with augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Define the batch size and target image size
batch_size = 32
img_height, img_width = 224, 224

# Define the train and test data generators
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(img_height, img_width),
                                               batch_size=batch_size,
                                               class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(img_height, img_width),
                                             batch_size=batch_size,
                                             class_mode='binary')

# Define the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model for 10 epochs
epochs = 5
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

model.save("/content/drive/MyDrive/trained_model_02.h5")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached jax-0.4.12-py3-none-any.whl
  Using cached jaxlib-0.4.12-cp310-cp310-manylinux2014_x86_64.whl (71.4 MB)
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.10+cuda11.cudnn86
    Uninstalling jaxlib-0.4.10+cuda11.cudnn86:
      Successfully uninstalled jaxlib-0.4.10+cuda11.cudnn86
  Attempting uninstall: jax
    Found existing installation: jax 0.4.10
    Uninstalling jax-0.4.10:
      Successfully uninstalled jax-0.4.10
Found 10442 images belonging to 2 classes.
Found 1248 images belonging to 2 classes.
Epoch 1/5
170/327 [==============>...............] - ETA: 20:30 - loss: 0.4243 - accuracy: 0.8219

In [ ]:
import tensorflow as tf

# Load the trained model from the h5 file
model_filename = "/content/drive/MyDrive/trained_model.h5"
model = tf.keras.models.load_model(model_filename)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the converted model to a .tflite file
tflite_model_filename = "/content/drive/MyDrive/new_model.tflite"
with open(tflite_model_filename, 'wb') as f:
    f.write(tflite_model)

print("Model converted and saved successfully!")



Model converted and saved successfully!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/trained_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
lite_model = converter.convert()

with open("/content/drive/MyDrive/lite_model.tflite", "wb") as f:
    f.write(lite_model)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')